In [5]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tensorflow as tf
import random

/Users/jmangia/anaconda2/envs/SwiftNLC/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
# import our chat-bot intents file
import json
with open('dataset.json') as json_data:
    intents = json.load(json_data)

In [8]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents utterances
for intent in intents['intents']:
    for pattern in intent['utterances']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['intent']))
        # add to our classes list
        if intent['intent'] not in classes:
            classes.append(intent['intent'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(classes), "classes", classes)
print (len(words), "unique stemmed words")
print (len(documents), "documents")
print (documents[0])
print (documents[1])




(25, 'classes', [u'about_VA', u'capabilities', u'compound_questions', u'decision_replies', u'goodbyes', u'greetings', u'improving_system', u'information_request', u'interface_interactions', u'interface_issues', u'locate_amenity', u'navigation', u'negative_reaction', u'not_specified', u'out_of_scope', u'phone', u'positive_reaction', u'selections', u'system_reliance', u'traffic_update', u'turn_down', u'turn_off', u'turn_on', u'turn_up', u'weather'])
(1320, 'unique stemmed words')
(2520, 'documents')
([u'hey', u',', u'now', u'it', u"'s", u'raining', u',', u'you', u'said', u'you', u'did', u"n't", u'know', u'anything', u'about', u'that', u'.', u'Are', u'you', u'a', u'liar', u'?'], u'system_reliance')
([u'do', u'you', u'tell', u'the', u'truth'], u'system_reliance')


In [15]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each intent and '1' for current intent
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

print (len(bag), "bags")    
print (len(training), "trainings")    

    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

(1320, 'bags')
(2520, 'trainings')


In [16]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [19]:
p = bow("I'm looking for pharmacy", words)
print (p)
print (classes)

[0 0 0 ... 0 0 0]
[u'about_VA', u'capabilities', u'compound_questions', u'decision_replies', u'goodbyes', u'greetings', u'improving_system', u'information_request', u'interface_interactions', u'interface_issues', u'locate_amenity', u'navigation', u'negative_reaction', u'not_specified', u'out_of_scope', u'phone', u'positive_reaction', u'selections', u'system_reliance', u'traffic_update', u'turn_down', u'turn_off', u'turn_on', u'turn_up', u'weather']


In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


Using TensorFlow backend.


In [25]:
model = Sequential()
model.add(Dense(8, input_dim=len(train_x[0]), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 10568     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 25)                225       
Total params: 10,865
Trainable params: 10,865
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [27]:
model.fit(train_x, train_y, epochs=40)

AttributeError: 'list' object has no attribute 'ndim'